**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [55]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [56]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [57]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
        
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act here determines action that will be taken by the agent based on current state where it is at. The function act behaves more like the exploration-exploitation arm of the agent where it picks a random action from all the posibble actions at 
hand if it is still training. 

The standard way to get exploration is to introduce an additional term, epsilon. We then randomly generate a value, and if that value is less than epsilon, a random action is chosen, otherwise we follow the normal tactic of choosing the max Q value, in other words we use the notion of exploitation.

$$ a^{*} = argmax_{a} Q^{\pi} (s , a) $$

Based on condition of epsilon it will explore or exploit the curent policy. The epsilon here acts as the measure that decides the tradeoff between exploration and exploiation. Basically this is implementing the epsilon-greedy approach by fixing an epsilon to decide the exploration the training model will do in order to avoid getting stuck at local optimum, here in our code we have defined the epsoilon to be 0.1 and whenever a random number is less than equal to 0.1 we explore and take a random path. If we increase or decrease epsilon it will increase or decrease our exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [58]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [59]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position is the array that contains the position of the rat on the board at a given time. The array is updated for each call of the function act by setting to zeros all the elements. There are two -1 cells thickness on the borders, reprenting where the rat can't go. That's why we added +4 to the grid size, allowing some place for these borders.The goal of the array is to keep track of the position of the mouse and give information about its playground.

board is the array that consists of the board on which the rat moves around. It is a matrix used to define the environment. Each of it cells is either neutral or represents the presence of the cheese or the poisonous cells. There informations represents the reward of each action and are represented by respectively 0, 0.5 and -1. The distribution of the reward (0.5 and -1) follows a binomial law that is defined is the reset function.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [60]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a 
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [61]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
        # The agent performs an action
            action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

        # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        
        ##### FILL IN HERE
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [62]:
# Initialize the game
env = Environment(grid_size=size, max_time=2000,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 19.0/39.0. Average score (-20.0)
Win/lose count 26.5/35.0. Average score (-14.25)
Win/lose count 28.0/39.0. Average score (-13.166666666666666)
Win/lose count 26.5/32.0. Average score (-11.25)
Win/lose count 28.0/35.0. Average score (-10.4)
Win/lose count 29.5/30.0. Average score (-8.75)
Win/lose count 25.5/32.0. Average score (-8.428571428571429)
Win/lose count 22.5/35.0. Average score (-8.9375)
Win/lose count 27.5/33.0. Average score (-8.555555555555555)
Win/lose count 28.5/27.0. Average score (-7.55)
Final score: -7.55


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




By definition of the action-value of a policy $\pi$, we have :

$Q^\pi(s,a)$
$Q^\pi(s,a) = E_\pi \Big[ \overset{\infty}{\underset{k=0}{\sum}} \gamma^k r_{t+k} \Big| s_t = s, a_t = a \Big]$

$= E\Big[r(s,a) + \overset{\infty}{\underset{k=1}{\sum}} \gamma^k r_{t+k} \Big| s_t = s, a_t = a\Big]$

$ =  r(s,a) + \gamma E\left[E\left[\overset{\infty}{\underset{k=0}{\sum}}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a'\right]|s_t = s, a_t = a\right]$

$ = r(s,a) + \gamma \underset{(s',a')}{\sum}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ))*E[\overset{\infty}{\underset{k=0}{\sum}}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']$

$ =  \underset{(s',a')}{\sum} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \underset{(s',a')}{\sum}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ))E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']$

$= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$

$~$
$~$

$Q^*(s,a)$
$Q^*(s,a) = \underset{(a',\pi')}{\max}\left[E_{(s',a')\sim p(.|s,a)}\left(r(s,a)+\gamma Q^{(a',\pi')}(s',a')\right)\right] $

$ = \underset{(a',\pi')}{\max}\left[r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right]$

$= r(s,a) + \gamma * \underset{(a',\pi')}{\max}\left[ \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{(a',\pi')}(s',a')\right] $

$~$ $~$

Rewriting $Q^{*}(s,a)$ $Q^{*}(s,a) = r(s,a) + \gamma  \overset{\infty}{\underset{k=0}{\sum}}(p(s_{t+1}=s' |s_t = s, a_t = a )\underset{a'}{\max}\underset{\pi'}{\max}Q^{\pi'}(s',a') $ 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [63]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        id_random = np.random.randint(len(self.memory))
        return self.memory[id_random]

***
The pipeline we will use for training is given below:

In [64]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [65]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        actions = self.model.predict(np.expand_dims(s, 0))
        id_max = np.argmax(actions)
        return id_max

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            actions = self.model.predict(np.expand_dims(s_, 0))
            value_max_action = np.max(actions)
            
            target_q[i] = actions
            
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount * value_max_action        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(512, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [66]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/010 | Loss 0.0062 | Win/lose count 7.5/10.0 (-2.5)
Epoch 001/010 | Loss 0.0064 | Win/lose count 9.0/7.0 (2.0)
Epoch 002/010 | Loss 0.0042 | Win/lose count 9.0/9.0 (0.0)
Epoch 003/010 | Loss 0.0022 | Win/lose count 9.0/8.0 (1.0)
Epoch 004/010 | Loss 0.0034 | Win/lose count 9.5/5.0 (4.5)
Epoch 005/010 | Loss 0.0256 | Win/lose count 11.5/6.0 (5.5)
Epoch 006/010 | Loss 0.0045 | Win/lose count 9.5/8.0 (1.5)
Epoch 007/010 | Loss 0.0017 | Win/lose count 8.0/9.0 (-1.0)
Epoch 008/010 | Loss 0.0015 | Win/lose count 13.0/6.0 (7.0)
Epoch 009/010 | Loss 0.0026 | Win/lose count 17.5/7.0 (10.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [71]:
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout
from keras.layers.core import Flatten

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (2, 2), input_shape=(5,5,self.n_state,), activation='relu'))
        model.add(Conv2D(32, (2, 2), activation='relu'))
        model.add(Dropout(rate=0.2))
        model.add(Flatten())
        # model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='sigmoid'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [68]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1size, epsilon = 0.5, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/010 | Loss 0.0047 | Win/lose count 2.5/3.0 (-0.5)
Epoch 001/010 | Loss 0.0035 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/010 | Loss 0.0055 | Win/lose count 4.5/8.0 (-3.5)
Epoch 003/010 | Loss 0.0004 | Win/lose count 7.5/3.0 (4.5)
Epoch 004/010 | Loss 0.0029 | Win/lose count 3.5/2.0 (1.5)
Epoch 005/010 | Loss 0.0024 | Win/lose count 2.5/3.0 (-0.5)
Epoch 006/010 | Loss 0.0026 | Win/lose count 5.0/4.0 (1.0)
Epoch 007/010 | Loss 0.0141 | Win/lose count 3.5/4.0 (-0.5)
Epoch 008/010 | Loss 0.0091 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/010 | Loss 0.0069 | Win/lose count 8.5/5.0 (3.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [72]:
epochs_test = 30

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=500, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.5, memory_size=500, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 13.0/11.0. Average score (2.0)
Win/lose count 13.0/10.0. Average score (2.5)
Win/lose count 9.5/13.0. Average score (0.5)
Win/lose count 3.0/5.0. Average score (-0.125)
Win/lose count 4.5/4.0. Average score (0.0)
Win/lose count 8.5/9.0. Average score (-0.08333333333333333)
Win/lose count 6.5/6.0. Average score (0.0)
Win/lose count 13.5/10.0. Average score (0.4375)
Win/lose count 20.5/13.0. Average score (1.2222222222222223)
Win/lose count 10.5/7.0. Average score (1.45)
Win/lose count 12.0/7.0. Average score (1.7727272727272727)
Win/lose count 2.5/3.0. Average score (1.5833333333333333)
Win/lose count 9.5/11.0. Average score (1.3461538461538463)
Win/lose count 11.0/7.0. Average score (1.5357142857142858)
Win/lose count 18.5/11.0. Average score (1.9333333333333333)
Win/lose count 9.0/9.0. Average score (1.8125)
Win/lose count 9.0/3.0. Average score (2.0588235294117645)
Win/lose count 12.0/11.0. Average score (2.0)
Win/lose count 15.5/9.0. Average score (2.2

In [73]:
HTML(display_videos('cnn_test0.mp4'))

In [74]:
HTML(display_videos('fc_test0.mp4'))

Comment on performance of fc_model and CNN: The results given by CNN model are a bit better than fc_model. Although in both cases he agent gets stuck and the exploration becomes non-existent.


By changing the temperature we are actually changing the probability of the binomial low. We notice that we improve the final score, but it is biased because by doing so we increase the probability of having cheese and we lower the probility of poisonious cells


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [75]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(np.float(agent.get_epsilon())*0.995)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
             
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [79]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.3, grid_size=size,  epsilon = 1, memory_size=16000, batch_size = 32,n_state=3)
train_explore(agent, env, 32, prefix='cnn_train_explore')

Epoch 000/032 | Loss 0.0044 | Win/lose count 9.0/14.229999999999974 (-5.229999999999974)
Epoch 001/032 | Loss 0.0051 | Win/lose count 1.49/7.829999999999962 (-6.339999999999962)
Epoch 002/032 | Loss 0.0160 | Win/lose count 2.95/1.8900000000000015 (1.0599999999999987)
Epoch 003/032 | Loss 0.0030 | Win/lose count 4.44/6.829999999999962 (-2.3899999999999615)
Epoch 004/032 | Loss 0.0168 | Win/lose count 2.4699999999999998/3.939999999999971 (-1.4699999999999713)
Epoch 005/032 | Loss 0.0162 | Win/lose count 1.96/2.9599999999999804 (-0.9999999999999805)
Epoch 006/032 | Loss 0.0079 | Win/lose count 3.9400000000000004/1.9000000000000015 (2.039999999999999)
Epoch 007/032 | Loss 0.0044 | Win/lose count 10.290000000000003/1.7800000000000014 (8.510000000000002)
Epoch 008/032 | Loss 0.0128 | Win/lose count 2.45/1.9600000000000015 (0.48999999999999866)
Epoch 009/032 | Loss 0.0027 | Win/lose count 3.4400000000000004/3.939999999999981 (-0.49999999999998046)
Epoch 010/032 | Loss 0.0104 | Win/lose count 

In [80]:
## Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 2.9400000000000004/1.9500000000000015. Average score (0.9899999999999989)
Win/lose count 2.45/1.9600000000000015. Average score (0.7399999999999988)
Win/lose count 4.410000000000001/1.9000000000000015. Average score (1.3299999999999992)
Win/lose count 2.95/1.9400000000000015. Average score (1.2499999999999991)
Win/lose count 2.45/1.9600000000000015. Average score (1.097999999999999)
Win/lose count 9.320000000000002/1.8200000000000014. Average score (2.164999999999999)
Win/lose count 4.410000000000001/1.9100000000000015. Average score (2.2128571428571417)
Win/lose count 3.920000000000001/1.9300000000000015. Average score (2.1849999999999987)
Win/lose count 0/3.0099999999999794. Average score (1.6077777777777789)
Win/lose count 6.870000000000002/2.8699999999999823. Average score (1.8470000000000026)
Win/lose count 6.370000000000002/1.8800000000000014. Average score (2.0872727272727296)
Win/lose count 3.9300000000000006/1.9200000000000015. Average score (2.0808333333333353)

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***